In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import math
import matplotlib
#matplotlib.use("Agg")
import matplotlib.image as mpimg
import glob
from data_utils import load_data, rgb2gray
import matplotlib.pyplot as plt
from models import complex_pokemon_model2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
Y_train, Y_test = load_data()

loading 0
file_name ../val_256/Places365_val_00011907.jpg is invalid
file_name ../val_256/Places365_val_00006664.jpg is invalid
file_name ../val_256/Places365_val_00015426.jpg is invalid
file_name ../val_256/Places365_val_00004517.jpg is invalid
(998, 256, 256, 3)
Shape of data_train, data_test: (798, 256, 256, 3) (200, 256, 256, 3)


In [ ]:
tf.reset_default_graph()

Y = tf.placeholder(tf.uint8 ,[None , 256,256,3])
is_training = tf.placeholder(tf.bool)

def show_images(images,gray = False):
    if(gray):
        plt.imshow(images,cmap = plt.get_cmap('gray'))
    else :
        plt.imshow(images);
    plt.show()

print (Y_train.shape)

with tf.variable_scope("model") as scope:
    hsv_out,rgb_out,hsv_in,rgb_in = complex_pokemon_model2(Y,is_training)
    loss = tf.reduce_mean(tf.square(hsv_out-hsv_in))
    optimiser = tf.train.AdamOptimizer(3e-3)
    train_step = optimiser.minimize(loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_size = 12
epoch_number = 0
def train_epoch(target):
    sizee = target.shape[0]
    for i in range(sizee//batch_size):
        feed_dict = {Y: (target[i*batch_size:batch_size*i+batch_size]),is_training:True}
        losses,_,rgb_o,rgb_i = sess.run([loss,train_step,rgb_out,rgb_in],feed_dict)
        print (losses)
        if(i%100 == 0):
            
            show_images(rgb_o[5])
            show_images(rgb_i[5])

def val_result(data, target):
    sizee = data.shape[0]
    for i in range(sizee//batch_size):
        feed_dict = {Y:  (target[i*batch_size:batch_size*i+batch_size]),
                     X: data[i*batch_size:batch_size*i+batch_size],is_training:False}
        losses = sess.run([loss_test,rgb_out],feed_dict)
        if(i%5 ==0):
            print (losses)

num_epochs = 5
for i in range(num_epochs):
    epoch_number = i
    train_epoch(Y_train)
    print ("Next Epoch",i)
    
    #val_result(X_test, Y_test)

(798, 256, 256, 3)
